Copyright **`(c)`** 2024 Giovanni Squillero `<giovanni.squillero@polito.it>`  
[`https://github.com/squillero/computational-intelligence`](https://github.com/squillero/computational-intelligence)  
Free under certain conditions — see the [`license`](https://github.com/squillero/computational-intelligence/blob/master/LICENSE.md) for details.  

In [1]:
import logging
from itertools import combinations
import pandas as pd
import numpy as np
from geopy.distance import geodesic
import networkx as nx
from tqdm.auto import tqdm

from icecream import ic

logging.basicConfig(level=logging.DEBUG)

In [2]:
def read_csv(country_name):
    CITIES = pd.read_csv(f'cities/{country_name}.csv', header=None, names=['name', 'lat', 'lon'])
    DIST_MATRIX = np.zeros((len(CITIES), len(CITIES)))
    for c1, c2 in combinations(CITIES.itertuples(), 2):
        DIST_MATRIX[c1.Index, c2.Index] = DIST_MATRIX[c2.Index, c1.Index] = geodesic(
            (c1.lat, c1.lon), (c2.lat, c2.lon)
        ).km
    CITIES.head()
    return CITIES, DIST_MATRIX

## Lab2 - TSP

https://www.wolframcloud.com/obj/giovanni.squillero/Published/Lab2-tsp.nb

In [3]:
def tsp_cost(tsp, CITIES, DIST_MATRIX):
    if tsp[0] != tsp[-1]:
        tsp = np.append(tsp, tsp[0])
    assert tsp[0] == tsp[-1]
    assert set(tsp) == set(range(len(CITIES)))

    tot_cost = 0
    for c1, c2 in zip(tsp, tsp[1:]):
        tot_cost += DIST_MATRIX[c1, c2]
    return tot_cost

In [4]:
def scrambleMutation(v1, B=1):
    """
    [1 2 3 4 5 6 7 8 9]
    [1 2 8 4 5 3 6 7 9]

    B is a coefficient related to strength of mutation. If B=1, there might be at most len(v1) can be occur
    """
    mutation_size = np.random.randint(low=2,high=len(v1)/B)
    chosen_ind = np.random.choice(len(v1), size=mutation_size, replace=False)
    chosen_elements = v1[chosen_ind]
    shuffled_vector = chosen_elements.copy()
    np.random.shuffle(shuffled_vector)
    v1[chosen_ind] = shuffled_vector

    return v1
    

In [5]:
def top_n_sol(population_arr, CITIES, DIST_MATRIX, K=1):
    cost_arr = np.zeros((population_arr.shape[0]))
    for i in range(population_arr.shape[0]):
        cost_arr[i] = tsp_cost(population_arr[i], CITIES, DIST_MATRIX)
    
    rankings = np.argsort(cost_arr)

    return population_arr[rankings][:K]


In [6]:
def cycleXover(v1,v2):
    child = np.zeros(v1.shape).astype(int)
    l1_l2 = np.random.choice(range(0, v1.shape[0]), size=2, replace=False)

    segment_from_p1, loci_from_p1 = v1[np.min(l1_l2): np.max(l1_l2)], np.arange(np.min(l1_l2), np.max(l1_l2))

    v2_ind = 0
    child[loci_from_p1] = segment_from_p1

    # Fill the rest
    for child_ind in range(len(child)):
        if child_ind in loci_from_p1:
            continue
        while v2[v2_ind] in segment_from_p1:
            v2_ind += 1
            if v2_ind > len(v2):
                return child
        child[child_ind] = v2[v2_ind]
        v2_ind += 1
        
    return child

In [7]:
def parent_selection(population, CITIES, DIST_MATRIX):
    """
    This function chooses parents with logarithmic probability.
    """
    ex = np.arange(population.shape[0]-1)
    # Define inverted logarithmic probabilities
    log_probs = np.log1p(max(ex) - ex)**5 # Inverting so that lower values have higher probabilities
    log_probs /= log_probs.sum()        # Normalize to create a probability distribution

    # Select two unique elements with logarithmic probability
    selected_elements = np.random.choice(ex, size=2, replace=False, p=log_probs)
    population = top_n_sol(population, CITIES, DIST_MATRIX, K=population.shape[1])
    return population[selected_elements[0]], population[selected_elements[1]]

## Only Scramble Mutation Algorithm

In [8]:
def PopulationScramble(CITIES, DIST_MATRIX, NUM_STEP, POPULATION_SIZE):
    city_population = np.array([np.random.choice(len(CITIES), size=(len(CITIES)), replace=False) for _ in range(POPULATION_SIZE)])
    curr_best = top_n_sol(city_population, CITIES, DIST_MATRIX,1)[0]

    print(f"Random solution is initialized. Cost: {tsp_cost(curr_best, CITIES, DIST_MATRIX)}")

    for i in tqdm(range(NUM_STEP)):
        offsprings = np.zeros(city_population.shape)

        for i in range(city_population.shape[0]):
            parent = city_population[i]
            child = parent.copy()
            child = scrambleMutation(child)
            offsprings[i] = child
        
        everybody = np.vstack((city_population, offsprings)).astype(int)
        new_generation = top_n_sol(everybody, CITIES, DIST_MATRIX, K=POPULATION_SIZE)
        city_population = new_generation
    
    curr_sol = city_population[0]


    print(f"Current solution is with a cost of {tsp_cost(curr_sol, CITIES, DIST_MATRIX)}")



## Only Cycle Crossover Algorithm

In [9]:
def PopulationCycle(CITIES, DIST_MATRIX,NUM_STEP, POPULATION_SIZE, NUM_OFFSPRINGS):
    city_population = np.array([np.random.choice(len(CITIES), size=(len(CITIES)), replace=False) for _ in range(POPULATION_SIZE)])
    curr_best = top_n_sol(city_population, CITIES, DIST_MATRIX,1)[0]


    print(f"Random solution is initialized. Cost: {tsp_cost(curr_best, CITIES, DIST_MATRIX)}")

    for i in tqdm(range(NUM_STEP)):
        offsprings = np.zeros((NUM_OFFSPRINGS, city_population.shape[1]))

        for i in range(NUM_OFFSPRINGS):
            parents = parent_selection(city_population, CITIES, DIST_MATRIX)
            child = parents[1].copy()
            child = cycleXover(parents[0], parents[1])
            offsprings[i] = child
        
        everybody = np.vstack((city_population, offsprings)).astype(int)
        new_generation = top_n_sol(everybody, CITIES, DIST_MATRIX, K=POPULATION_SIZE)
        city_population = new_generation
    
    curr_sol = city_population[0]


    print(f"Current solution is with a cost of {tsp_cost(curr_sol, CITIES, DIST_MATRIX)}")

## Scramble Mutation + Cycle Crossever Algorithm

In [10]:
NUM_STEP = 10000
POPULATION_SIZE = 10
NUM_OFFSPRINGS = 10
P_MUTATION = 0.1
P_XOVER = 1-P_MUTATION

def PopulationHybrid(CITIES, DIST_MATRIX,NUM_STEP, POPULATION_SIZE, NUM_OFFSPRINGS,P_MUTATION):
    city_population = np.array([np.random.choice(len(CITIES), size=(len(CITIES)), replace=False) for _ in range(POPULATION_SIZE)])
    curr_best = top_n_sol(city_population,CITIES, DIST_MATRIX,1)[0]


    print(f"Random solution is initialized. Cost: {tsp_cost(curr_best, CITIES, DIST_MATRIX)}")

    for i in tqdm(range(NUM_STEP)):
        if np.random.random() < P_MUTATION:
            offsprings = np.zeros((NUM_OFFSPRINGS, city_population.shape[1]))

            for i in range(NUM_OFFSPRINGS):
                parents = parent_selection(city_population, CITIES, DIST_MATRIX)
                child = parents[1].copy()
                child = cycleXover(parents[0], parents[1])
                offsprings[i] = child
            
            everybody = np.vstack((city_population, offsprings)).astype(int)
            new_generation = top_n_sol(everybody, CITIES, DIST_MATRIX, K=POPULATION_SIZE)
            city_population = new_generation
        else:
            offsprings = np.zeros(city_population.shape)

            for i in range(city_population.shape[0]):
                parent = city_population[i]
                child = parent.copy()
                child = scrambleMutation(child)
                offsprings[i] = child
            
            everybody = np.vstack((city_population, offsprings)).astype(int)
            new_generation = top_n_sol(everybody, CITIES, DIST_MATRIX, K=POPULATION_SIZE)
            city_population = new_generation
          
    curr_sol = city_population[0]

    print(f"Current solution is with a cost of {tsp_cost(curr_sol, CITIES, DIST_MATRIX)}")

## Test Runs

### Vanuatu

In [11]:
CITIES, DIST_MATRIX = read_csv("vanuatu")
NUM_STEP = 10000
POPULATION_SIZE = 10
NUM_OFFSPRINGS = 10
PopulationScramble(CITIES, DIST_MATRIX, NUM_STEP, POPULATION_SIZE)
print()

NUM_STEP = 10000
POPULATION_SIZE = 10
NUM_OFFSPRINGS = 10
PopulationCycle(CITIES, DIST_MATRIX,NUM_STEP, POPULATION_SIZE, NUM_OFFSPRINGS)
print()

NUM_STEP = 10000
POPULATION_SIZE = 10
NUM_OFFSPRINGS = 10
P_MUTATION = 0.8
P_XOVER = 1-P_MUTATION

PopulationHybrid(CITIES, DIST_MATRIX,NUM_STEP, POPULATION_SIZE, NUM_OFFSPRINGS,P_MUTATION)
print()

P_MUTATION = 0.2
P_XOVER = 1-P_MUTATION

PopulationHybrid(CITIES, DIST_MATRIX,NUM_STEP, POPULATION_SIZE, NUM_OFFSPRINGS,P_MUTATION)

Random solution is initialized. Cost: 1640.2978447123357


  0%|          | 0/10000 [00:00<?, ?it/s]

Current solution is with a cost of 1345.544956473311

Random solution is initialized. Cost: 1475.7794291837738


  0%|          | 0/10000 [00:00<?, ?it/s]

Current solution is with a cost of 1472.198639920995

Random solution is initialized. Cost: 1558.9987568276256


  0%|          | 0/10000 [00:00<?, ?it/s]

Current solution is with a cost of 1345.5449564733112

Random solution is initialized. Cost: 1595.498382618305


  0%|          | 0/10000 [00:00<?, ?it/s]

Current solution is with a cost of 1345.544956473311


### Italy

In [12]:
CITIES, DIST_MATRIX = read_csv("italy")
NUM_STEP = 10000
POPULATION_SIZE = 10
NUM_OFFSPRINGS = 10
PopulationScramble(CITIES, DIST_MATRIX, NUM_STEP, POPULATION_SIZE)
print()

NUM_STEP = 10000
POPULATION_SIZE = 10
NUM_OFFSPRINGS = 10
PopulationCycle(CITIES, DIST_MATRIX,NUM_STEP, POPULATION_SIZE, NUM_OFFSPRINGS)
print()

NUM_STEP = 10000
POPULATION_SIZE = 10
NUM_OFFSPRINGS = 10
P_MUTATION = 0.8
P_XOVER = 1-P_MUTATION

PopulationHybrid(CITIES, DIST_MATRIX,NUM_STEP, POPULATION_SIZE, NUM_OFFSPRINGS,P_MUTATION)
print()

P_MUTATION = 0.2
P_XOVER = 1-P_MUTATION

PopulationHybrid(CITIES, DIST_MATRIX,NUM_STEP, POPULATION_SIZE, NUM_OFFSPRINGS,P_MUTATION)

Random solution is initialized. Cost: 16936.160768931724


  0%|          | 0/10000 [00:00<?, ?it/s]

Current solution is with a cost of 6003.077652431114

Random solution is initialized. Cost: 16203.79297172712


  0%|          | 0/10000 [00:00<?, ?it/s]

Current solution is with a cost of 15445.181747080811

Random solution is initialized. Cost: 17222.927465255976


  0%|          | 0/10000 [00:00<?, ?it/s]

Current solution is with a cost of 7412.864129135289

Random solution is initialized. Cost: 17969.91733844916


  0%|          | 0/10000 [00:00<?, ?it/s]

Current solution is with a cost of 6792.071074456272


### US

In [13]:
CITIES, DIST_MATRIX = read_csv("us")
NUM_STEP = 10000
POPULATION_SIZE = 10
NUM_OFFSPRINGS = 10
PopulationScramble(CITIES, DIST_MATRIX, NUM_STEP, POPULATION_SIZE)
print()

NUM_STEP = 10000
POPULATION_SIZE = 10
NUM_OFFSPRINGS = 10
PopulationCycle(CITIES, DIST_MATRIX,NUM_STEP, POPULATION_SIZE, NUM_OFFSPRINGS)
print()

NUM_STEP = 10000
POPULATION_SIZE = 10
NUM_OFFSPRINGS = 10
P_MUTATION = 0.8
P_XOVER = 1-P_MUTATION

PopulationHybrid(CITIES, DIST_MATRIX,NUM_STEP, POPULATION_SIZE, NUM_OFFSPRINGS,P_MUTATION)
print()

P_MUTATION = 0.2
P_XOVER = 1-P_MUTATION

PopulationHybrid(CITIES, DIST_MATRIX,NUM_STEP, POPULATION_SIZE, NUM_OFFSPRINGS,P_MUTATION)

Random solution is initialized. Cost: 642976.1252221419


  0%|          | 0/10000 [00:00<?, ?it/s]

Current solution is with a cost of 376959.58142718324

Random solution is initialized. Cost: 640704.5756262504


  0%|          | 0/10000 [00:00<?, ?it/s]

Current solution is with a cost of 575475.0662276574

Random solution is initialized. Cost: 626594.6940289958


  0%|          | 0/10000 [00:00<?, ?it/s]

Current solution is with a cost of 353738.7889857448

Random solution is initialized. Cost: 608285.7225008543


  0%|          | 0/10000 [00:00<?, ?it/s]

Current solution is with a cost of 310356.83249965


### China

In [14]:
CITIES, DIST_MATRIX = read_csv("china")
NUM_STEP = 10000
POPULATION_SIZE = 10
NUM_OFFSPRINGS = 10
PopulationScramble(CITIES, DIST_MATRIX, NUM_STEP, POPULATION_SIZE)
print()

NUM_STEP = 10000
POPULATION_SIZE = 10
NUM_OFFSPRINGS = 10
PopulationCycle(CITIES, DIST_MATRIX,NUM_STEP, POPULATION_SIZE, NUM_OFFSPRINGS)
print()

NUM_STEP = 10000
POPULATION_SIZE = 10
NUM_OFFSPRINGS = 10
P_MUTATION = 0.8
P_XOVER = 1-P_MUTATION

PopulationHybrid(CITIES, DIST_MATRIX,NUM_STEP, POPULATION_SIZE, NUM_OFFSPRINGS,P_MUTATION)
print()

P_MUTATION = 0.2
P_XOVER = 1-P_MUTATION

PopulationHybrid(CITIES, DIST_MATRIX,NUM_STEP, POPULATION_SIZE, NUM_OFFSPRINGS,P_MUTATION)

Random solution is initialized. Cost: 943644.530791301


  0%|          | 0/10000 [00:00<?, ?it/s]

Current solution is with a cost of 724506.7601957914

Random solution is initialized. Cost: 958632.1539599255


  0%|          | 0/10000 [00:00<?, ?it/s]

Current solution is with a cost of 920172.0514125223

Random solution is initialized. Cost: 940714.6609472723


  0%|          | 0/10000 [00:00<?, ?it/s]

Current solution is with a cost of 692287.0202895127

Random solution is initialized. Cost: 936060.7834084854


  0%|          | 0/10000 [00:00<?, ?it/s]

Current solution is with a cost of 646924.1069130485


### Russia

In [15]:
CITIES, DIST_MATRIX = read_csv("russia")
NUM_STEP = 10000
POPULATION_SIZE = 10
NUM_OFFSPRINGS = 10
PopulationScramble(CITIES, DIST_MATRIX, NUM_STEP, POPULATION_SIZE)
print()

NUM_STEP = 10000
POPULATION_SIZE = 10
NUM_OFFSPRINGS = 10
PopulationCycle(CITIES, DIST_MATRIX,NUM_STEP, POPULATION_SIZE, NUM_OFFSPRINGS)
print()

NUM_STEP = 10000
POPULATION_SIZE = 10
NUM_OFFSPRINGS = 10
P_MUTATION = 0.8
P_XOVER = 1-P_MUTATION

PopulationHybrid(CITIES, DIST_MATRIX,NUM_STEP, POPULATION_SIZE, NUM_OFFSPRINGS,P_MUTATION)
print()

P_MUTATION = 0.2
P_XOVER = 1-P_MUTATION

PopulationHybrid(CITIES, DIST_MATRIX,NUM_STEP, POPULATION_SIZE, NUM_OFFSPRINGS,P_MUTATION)

Random solution is initialized. Cost: 312111.93350161664


  0%|          | 0/10000 [00:00<?, ?it/s]

Current solution is with a cost of 138062.27574766215

Random solution is initialized. Cost: 321076.8597781184


  0%|          | 0/10000 [00:00<?, ?it/s]

Current solution is with a cost of 287453.0349327676

Random solution is initialized. Cost: 311300.6589765704


  0%|          | 0/10000 [00:00<?, ?it/s]

Current solution is with a cost of 159129.64927027095

Random solution is initialized. Cost: 322881.6127242792


  0%|          | 0/10000 [00:00<?, ?it/s]

Current solution is with a cost of 163905.92171973202
